In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib sklearn mediapipe

In [2]:
import cv2
from matplotlib import pyplot as plt
import os
import time
import mediapipe as mp
import numpy as np

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection (image,model):
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)
    return image , results

In [5]:
def draw_landmarks(image , results):
    mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION )
    mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS )
    mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS )
    mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS )

In [6]:
def draw_style_landmarks(image , results):
    mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION ,
                             mp_drawing.DrawingSpec(color=(80,110,10) , thickness=1 , circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,121) , thickness=1 , circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS  ,
                             mp_drawing.DrawingSpec(color=(80,22,10) , thickness=2 , circle_radius=4),
                            mp_drawing.DrawingSpec(color=(80,44,121) , thickness=2 , circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS ,
                             mp_drawing.DrawingSpec(color=(121,22,76) , thickness=2 , circle_radius=4),
                            mp_drawing.DrawingSpec(color=(121,44,250) , thickness=2 , circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS ,
                             mp_drawing.DrawingSpec(color=(121,22,76) , thickness=2 , circle_radius=4),
                            mp_drawing.DrawingSpec(color=(121,44,250) , thickness=2 , circle_radius=2)
                             )

In [7]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():

        ret,frame = cap.read()

        image,results = mediapipe_detection(frame,holistic)
        print(results)
        
        draw_style_landmarks(image , results)

        cv2.imshow('isaret Algılayıcı' , image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [9]:
pose = np.array([[res.x , res.y , res.z , res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x , res.y , res.z ] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x , res.y , res.z ] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
rh = np.array([[res.x , res.y , res.z ] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)

In [10]:
def extract_keypoints (results):
    pose = np.array([[res.x , res.y , res.z , res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x , res.y , res.z ] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x , res.y , res.z ] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x , res.y , res.z ] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lh, rh])

In [11]:
result_test = extract_keypoints(results)

In [12]:
np.save('0' , result_test)

In [13]:
DATA_PATH = os.path.join('MP_Data')

actions = np.array(['Hello' , 'Thanks' , 'iLoveyou'])

no_sequences = 30

sequence_lenght = 30

In [14]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH , action, str(sequence)))
        except:
            pass

In [14]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence = 0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_lenght):

                ret,frame = cap.read()

                image,results = mediapipe_detection(frame,holistic)
                print(results)

                draw_style_landmarks(image , results)
                
                if frame_num == 0 :
                    cv2.putText(image , 'STARTING COLLECTION' , (120,200) ,
                                cv2.FONT_HERSHEY_SIMPLEX , 1 , (0,255,0) , 4 , cv2.LINE_AA)
                    cv2.putText(image , 'Collecting frames for {} Video number {}'.format(action,sequence) , (15,12) ,
                                cv2.FONT_HERSHEY_SIMPLEX , 0.5 , (0,0,255) , 1 , cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image , 'Collecting frames for {} Video number {}'.format(action,sequence) , (15,12) ,
                                cv2.FONT_HERSHEY_SIMPLEX , 0.5 , (0,0,255) , 1 , cv2.LINE_AA)
                
                keypoints = extract_keypoints(results)
                
                npy_path = os.path.join(DATA_PATH , action , str(sequence) , str(frame_num))
                
                np.save(npy_path , keypoints)

                cv2.imshow('isaret Algılayıcı' , image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
label_map = {label:num for num , label in enumerate(actions)}

In [17]:
label_map

{'Hello': 0, 'Thanks': 1, 'iLoveyou': 2}

In [18]:
cap.release()
cv2.destroyAllWindows()

In [19]:
sequences , labels = [] ,[]

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_lenght):
            res = np.load(os.path.join(DATA_PATH , action , str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
X = np.array(sequences)

In [21]:
X.shape

(90, 30, 1662)

In [22]:
y = to_categorical(labels).astype(int)

In [23]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [24]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.05)

In [25]:
y_test.shape

(5, 3)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dense
from tensorflow.keras.callbacks import TensorBoard

In [27]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir= log_dir)

In [39]:
model = Sequential()
model.add(LSTM(64 , return_sequences=True , activation = 'relu' , input_shape = (30,1662)))
model.add(LSTM(128 , return_sequences=True , activation = 'relu'))
model.add(LSTM(64 , return_sequences=False , activation = 'relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [38]:
model.compile(optimizer='Adam' , loss='categorical_crossentropy' , metrics=['categorical_accuracy'])

NameError: name 'model' is not defined

In [31]:
model.fit(X_train , y_train , epochs=750 , callbacks=[tb_callback])

Epoch 1/750
3/3 [==============================] - 11s 1s/step - loss: 9.3094 - categorical_accuracy: 0.3647
Epoch 2/750
3/3 [==============================] - 3s 1s/step - loss: 165.9580 - categorical_accuracy: 0.3765
Epoch 3/750
3/3 [==============================] - 3s 1s/step - loss: 48.0438 - categorical_accuracy: 0.2824
Epoch 4/750
3/3 [==============================] - 4s 1s/step - loss: 27.9160 - categorical_accuracy: 0.2706
Epoch 5/750
3/3 [==============================] - 3s 1s/step - loss: 51.8651 - categorical_accuracy: 0.3294
Epoch 6/750
3/3 [==============================] - 3s 1s/step - loss: 123.1383 - categorical_accuracy: 0.3059
Epoch 7/750
3/3 [==============================] - 3s 1s/step - loss: 136.3888 - categorical_accuracy: 0.4353
Epoch 8/750


KeyboardInterrupt: 

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [33]:
res = model.predict(X_test)

1/1 [==============================] - 1s 634ms/step


In [34]:
actions[np.argmax(res[0])]

'Hello'

In [35]:
actions[np.argmax(y_test[0])]

'Hello'

In [36]:
del model

In [40]:
model.load_weights('isaret_algılayıcı.h5')

In [41]:
from sklearn.metrics import accuracy_score , multilabel_confusion_matrix

In [42]:
yhat = model.predict(X_train)

3/3 [==============================] - 1s 133ms/step


In [43]:
ytrue = np.argmax(y_train , axis=1).tolist()
yhat = np.argmax(yhat , axis=1).tolist()

In [44]:
multilabel_confusion_matrix(ytrue , yhat)

array([[[56,  0],
        [10, 19]],

       [[46, 10],
        [ 0, 29]],

       [[58,  0],
        [ 0, 27]]], dtype=int64)

In [45]:
accuracy_score(ytrue,yhat)

0.8823529411764706

In [46]:
colors = [(255,0,0) , (0,255,0) , (0,0,255)]

def prob_viz(res , actions , input_frame , colors):
    output_frame = input_frame.copy()
    for num , prob in enumerate(res):
        cv2.rectangle(output_frame , (0,60+num*40) , (int(prob*100) , 90+num*40) , colors[num] ,-1 )
        cv2.putText(output_frame , actions[num] , (0,85+num*40) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255,255,255) , 2 , cv2.LINE_AA)
        
    return output_frame

In [47]:
plt.imshow(prob_viz(res,actions,image,colors))

TypeError: only size-1 arrays can be converted to Python scalars

In [66]:
sequence = []
sentence = []
threshold = 0.5


cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():

        ret,frame = cap.read()

        image,results = mediapipe_detection(frame,holistic)
        print(results)
        
        draw_style_landmarks(image , results)
        
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence , axis=0))[0]
            print(actions[np.argmax(res)])
        
        if 1> 0.5:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
        
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        image = prob_viz(res , actions , image , colors)
            
            
        cv2.rectangle(image , (0,0) ,(640,40), (245,117,16) ,)
        cv2.putText(image , ' '.join(sentence) , (3,30) ,
                    cv2.FONT_HERSHEY_SIMPLEX , 1 , (0,0,0) , 2 , cv2.LINE_AA)

        cv2.imshow('Isaret Algilayici' , image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


TypeError: only size-1 arrays can be converted to Python scalars

In [67]:
cap.release()
cv2.destroyAllWindows()

In [57]:
res[np.argmax(res)] 

3